# 📘 Multi-Task Gaussian Process (Production Ready)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/benmola/OpenAD-lib/blob/main/notebooks/04_MTGP_Prediction_Full.ipynb)

Predicting **multiple AD outputs** (SCOD, VFA, Biogas) with **uncertainty quantification**.

**⚠️ This notebook matches `examples/05_mtgp_prediction.py` exactly**

---

## 📚 References
- **MTGP for AD**: [Dekhici et al. (2025) - LAPSE](https://psecommunity.org/LAPSE:2025.0155)

## 🔬 Gaussian Process Background

### What is a Gaussian Process?

A GP defines a **distribution over functions**:

$$f(x) \sim \mathcal{GP}(m(x), k(x, x'))$$

Where:
- $m(x)$ = mean function (usually 0)
- $k(x, x')$ = **kernel** function measuring similarity

### Why GPs for Biogas?

1. **Uncertainty Quantification** - Get confidence intervals for free!
2. **Data Efficient** - Work well with small datasets (50-200 samples)
3. **Non-parametric** - No assumptions about functional form

### Multi-Task Learning with LMC

**Problem:** Predict 3 correlated outputs (SCOD, VFA, Biogas)

**Solution:** Linear Model of Coregionalization (LMC)

$$f_t(x) = \sum_{q=1}^{Q} a_{t,q} \cdot u_q(x)$$

- $f_t$ = function for task $t$ (e.g., VFA prediction)
- $u_q$ = shared latent function $q$
- $a_{t,q}$ = weight (learned automatically)

**Key Insight:** VFA and Biogas are correlated → share information!

### Predictive Distribution

$$p(f_* | X_*, X, Y) = \mathcal{N}(\mu_*, \Sigma_*)$$

We get:
- **Mean prediction:** $\mu_*$
- **Uncertainty:** $\pm 2\sigma_*$ (95% confidence interval)

## 1️⃣ Setup

In [ ]:
# Install with ML dependencies (GPyTorch, PyTorch)
!pip install git+https://github.com/benmola/OpenAD-lib.git

import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:
    sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

print(f"Running in Colab: {IN_COLAB}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openad_lib.models.ml import MultitaskGP

print("✅ Imports successful!")

## 2️⃣ Load Multi-Output AD Data

**Dataset:** `sample_ad_process_data.csv`

**Inputs (5):**
- `time` - Day number
- `D` - Dilution rate (1/day)
- `SCODin` - Influent SCOD (g COD/L)
- `OLR` - Organic Loading Rate (g COD/L/day)
- `pH` - Reactor pH

**Outputs (3):** All correlated!
- `SCODout` - Effluent SCOD → waste
- `VFAout` - VFA concentration → process stability indicator
- `Biogas` - Biogas production → revenue

**Why predict all 3?** VFA ↑ often means Biogas ↓ (process inhibition)

In [ ]:
# Download for Colab
if IN_COLAB:
    !wget -q https://raw.githubusercontent.com/benmola/OpenAD-lib/main/src/openad_lib/data/sample_ad_process_data.csv
    data_path = 'sample_ad_process_data.csv'
else:
    base_path = os.path.dirname(os.getcwd())
    data_path = os.path.join(base_path, 'src', 'openad_lib', 'data', 'sample_ad_process_data.csv')

# Load
data = pd.read_csv(data_path)
print(f"📊 Loaded {len(data)} samples")
print(f"\nColumns: {list(data.columns)}")
data.head()

In [ ]:
# CRITICAL: Define columns explicitly (MUST match example)
input_cols = ['time', 'D', 'SCODin', 'OLR', 'pH']
output_cols = ['SCODout', 'VFAout', 'Biogas']

# Verify columns exist
missing_inputs = [c for c in input_cols if c not in data.columns]
missing_outputs = [c for c in output_cols if c not in data.columns]

if missing_inputs or missing_outputs:
    print(f"❌ Missing columns!")
    print(f"   Inputs: {missing_inputs}")
    print(f"   Outputs: {missing_outputs}")
else:
    print("✅ All columns found")
    
# Extract data
X = data[input_cols].values
Y = data[output_cols].values

print(f"\nInput shape: {X.shape} (5 features)")
print(f"Output shape: {Y.shape} (3 tasks)")

## 3️⃣ Train/Test Split Strategy

**Alternating Split** (not random!):

```
Original:  0  1  2  3  4  5  6  7  8  9 ...
Test:      ✓     ✓     ✓     ✓     ✓    ... (even indices)
Train:        ✓     ✓     ✓     ✓   ...    (odd indices)
```

**Why alternating?**
- Tests interpolation ability (realistic for AD)
- Preserves temporal distribution
- Avoids train/test distribution shift

**From reference paper:** This split maximizes GP's strength in smooth interpolation.

In [ ]:
# Alternating indices (MUST match example)
train_indices = np.arange(1, len(X), 2)  # [1, 3, 5, 7, ...]
test_indices = np.arange(0, len(X), 2)   # [0, 2, 4, 6, ...]

X_train, X_test = X[train_indices], X[test_indices]
Y_train, Y_test = Y[train_indices], Y[test_indices]

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"\nSplit ratio: {len(X_train)/(len(X_train)+len(X_test)):.1%} train")

## 4️⃣ MTGP Model Configuration

**Hyperparameters (tuned via validation):**

| Parameter | Value | Purpose |
|-----------|-------|----------|
| `num_tasks` | 3 | Number of outputs (SCOD, VFA, Biogas) |
| `num_latents` | 3 | Shared latent functions for LMC |
| `n_inducing` | 60 | Inducing points for scalability |
| `learning_rate` | 0.1 | Adam optimizer LR |
| `log_transform` | True | Handle positive-only outputs |

**Why these values?**
- `num_latents=3`: Each task gets its own latent + sharing
- `n_inducing=60`: ~50% of training data (good for ~100 samples)
- `log_transform`: Biogas/VFA can't be negative!

In [ ]:
# Initialize MTGP (MUST match example hyperparameters)
num_tasks = Y.shape[1]
print(f"🔧 Initializing MTGP with {num_tasks} tasks...")

mtgp = MultitaskGP(
    num_tasks=num_tasks,
    num_latents=min(3, num_tasks),  # Cap at 3 latent functions
    n_inducing=60,
    learning_rate=0.1,
    log_transform=True  # Critical for positive outputs!
)

print("✅ Model initialized")

## 5️⃣ Train MTGP Model

**Training Process:**
1. Optimize kernel hyperparameters (lengthscales, variance)
2. Learn task correlations (LMC weights $a_{t,q}$)
3. Update inducing point locations

**500 epochs** - typical for variational GP training

In [ ]:
print("🚀 Training MTGP (500 iterations)...\n")
mtgp.fit(X_train, Y_train, epochs=500, verbose=True)
print("\n✅ Training complete!")

## 6️⃣ Predict with Uncertainty

**GP provides 3 values for each prediction:**

1. **Mean** ($\mu_*$): Best estimate
2. **Lower bound** ($\mu_* - 2\sigma_*$): 2.5th percentile
3. **Upper bound** ($\mu_* + 2\sigma_*$): 97.5th percentile

**95% Confidence Interval = [Lower, Upper]**

**Uncertainty types:**
- **Aleatoric**: Data noise (irreducible)
- **Epistemic**: Model uncertainty (reduces with more data)

In [ ]:
print("🔮 Predicting on test set with uncertainty...")
mean, lower, upper = mtgp.predict(X_test, return_std=True)

print(f"\nPrediction shapes:")
print(f"  Mean: {mean.shape}")
print(f"  Lower (2.5%): {lower.shape}")
print(f"  Upper (97.5%): {upper.shape}")

# Average uncertainty width per task
for i, task in enumerate(output_cols):
    avg_width = (upper[:, i] - lower[:, i]).mean()
    print(f"  {task}: Avg CI width = {avg_width:.4f}")

## 7️⃣ Evaluate Performance

**Metrics per task:**
- **RMSE**: Prediction error magnitude
- **MAE**: Average absolute error
- **R²**: Variance explained (1.0 = perfect)

In [ ]:
metrics = mtgp.evaluate(X_test, Y_test, task_names=output_cols)

print("📊 MTGP Test Metrics:")
print("=" * 60)
for task, vals in metrics.items():
    print(f"{task:10s}: RMSE={vals['rmse']:.4f}, MAE={vals['mae']:.4f}, R²={vals['r2']:.4f}")
print("\n✅ These metrics should match examples/05_mtgp_prediction.py")

## 8️⃣ Visualize Predictions with Uncertainty

**Each subplot shows:**
- 🔵 **Blue dots** = Training data
- 🔴 **Red dots** = Test data (ground truth)
- ⬛ **Black line** = GP mean prediction
- 🌫️ **Gray band** = 95% confidence interval

**How to interpret:**
- **Narrow CI** = Model is confident (lots of nearby training data)
- **Wide CI** = Model is uncertain (sparse data region)

In [ ]:
plt.style.use('bmh')
fig, axes = plt.subplots(num_tasks, 1, figsize=(12, 4*num_tasks), sharex=True)

for i, (ax, task) in enumerate(zip(axes, output_cols)):
    # Training data (blue dots)
    ax.plot(X_train[:, 0], Y_train[:, i], 'o', 
            color='#2E86C1', markersize=5, alpha=0.5, label='Train Data')
    
    # Test data (red dots)
    ax.plot(X_test[:, 0], Y_test[:, i], 'o', 
            color='#E74C3C', markersize=6, alpha=0.7, label='Test Data (True)')
    
    # GP mean prediction (black line)
    ax.plot(X_test[:, 0], mean[:, i], '-', 
            color='black', linewidth=2, label='MTGP Mean')
    
    # 95% Confidence interval (gray band)
    ax.fill_between(X_test[:, 0], lower[:, i], upper[:, i],
                    color='gray', alpha=0.3, label='95% Confidence')
    
    ax.set_ylabel(task, fontsize=14, fontweight='bold')
    ax.set_title(f'{task} - R² = {metrics[task]["r2"]:.3f}', fontsize=14, pad=10)
    ax.legend(fontsize=11, loc='best')
    ax.grid(True, linestyle='--', alpha=0.7)

axes[-1].set_xlabel('Time (days)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 📝 Summary

This notebook demonstrated:

1. **Multi-Task GP** - Predicting 3 correlated outputs jointly
2. **LMC Framework** - Sharing information across tasks
3. **Uncertainty Quantification** - 95% confidence intervals
4. **Alternating Split** - Realistic interpolation evaluation

### 🎯 Key Advantages of MTGP

| Feature | MTGP | LSTM |
|---------|------|------|
| **Uncertainty** | ✅ Built-in CI | ❌ Needs ensembles |
| **Multi-output** | ✅ Correlated sharing | ❌ Independent |
| **Small data** | ✅ Works with 50-100 | ❌ Needs 500+ |
| **Interpretability** | ✅ Kernel inspection | ❌ Black box |

### 📚 When to Use MTGP?

✅ **Use MTGP when:**
- You need **uncertainty estimates** for decision-making
- You have **limited data** (<200 samples)
- Outputs are **correlated** (VFA ↔ Biogas)
- You need **confidence intervals** for safety-critical applications

❌ **Use LSTM instead when:**
- Strong temporal dependencies (long time lags)
- Large datasets (>1000 samples)
- Computational speed critical

### Next Steps

- Compare with [LSTM](03_LSTM_Prediction_Full.ipynb)
- Apply to [MPC Control](05_MPC_Control_Full.ipynb) with uncertainty
- See [ADM1](01_ADM1_Tutorial_Full.ipynb) for mechanistic modeling